# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 04:01:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   63C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2019년 LG Data**

In [5]:
lexicon_2019 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2019_lg.csv')

lexicon_2019.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,LG화학,51910,매일경제,2019010117,외국인 LG화학 사모은 까닭은,http://news.mk.co.kr/newsRead.php?year=2019&no...,\n\n\n 외국인 투자자가 지난해 말 LG화학을 집중 매수한 것으로 집계됐다. 한...,2019-01-02,17,351000,353500,335500,337000,227694,-0.028818,-1,0,외국인 투자자 지난해 화학 집중 매수 집계 한국 증시 팔자 행진 전기차 배터리 시장...


In [6]:
lexicon_2019[lexicon_2019['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2019.dropna(axis=0, inplace=True)
lexicon_2019 = lexicon_2019.reset_index(drop=True)
lexicon_2019.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2019 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2019['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2019['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2019['Tokenization'][x].split())):
                    if lexicon_2019['Tokenization'][x].split()[w] == list(set(lexicon_2019['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2019['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2019['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2019 = pos_lexicon_2019.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2019['date'][x], positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z])))

***2019-01-02 Cosine Similarity between <개선> & <향상> : 0.5878475904464722
***2019-01-02 Cosine Similarity between <개선> & <확충> : 0.5164448022842407
***2019-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2019-01-02 Cosine Similarity between <본격적인> & <본격> : 0.6104379892349243
***2019-01-02 Cosine Similarity between <장점> & <강점> : 0.5735523700714111
***2019-01-02 Cosine Similarity between <장점> & <특성> : 0.5244326591491699
***2019-01-02 Cosine Similarity between <증대> & <향상> : 0.6337747573852539
***2019-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2019-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2019-01-02 Cosine Similarity between <증대> & <확충> : 0.5278360247612
***2019-01-02 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2019-01-02 Cosine Similarity between <확보> & <확충> : 0.5350995659828186
***2019-01-02 Cosine Similarity between <장점> & <강점> : 0.5735523700714111
***2019-01-02 Cosine Similarity between <중요한> & <가장> 

In [11]:
pos_lexicon_2019

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2019-01-02,1,개선,향상,0.587848,1
1,2019-01-02,1,개선,확충,0.516445,1
2,2019-01-02,1,경신,기록,0.522991,1
3,2019-01-02,1,본격적인,본격,0.610438,2
4,2019-01-02,1,장점,강점,0.573552,1
...,...,...,...,...,...,...
8810,2020-01-02,1750,경신,기록,0.522991,2
8811,2020-01-02,1750,중요한,가장,0.537125,4
8812,2020-01-02,1750,추진,추진,1.000000,1
8813,2020-01-02,1750,추진,검토,0.511331,1


In [12]:
pos_lexicon_2019.to_csv('../../../../Code/Data/Test/Stock-Year/pos_lg_2019.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2019['news_word'])))

a

['구현',
 '정찬',
 '공유진',
 '쿠츠하르',
 '추진',
 '남호정',
 '뉴인텍뉴인텍',
 '요구',
 '언택트',
 '라니티닌',
 '소형모듈원전',
 '텅빈채',
 '컨설팅하거',
 '불황',
 '엠에스',
 '엑스퍼',
 '산단특위',
 '창저우시',
 '특징',
 '유지담',
 '향상',
 '나경환',
 '시나리오플래닝',
 '강세',
 '래깅효',
 '셀루메드셀루메드',
 '웃돌',
 '테슬론',
 '증대',
 '에비츠에너르',
 '가장',
 '중인',
 '진행',
 '자공시',
 '남경시',
 '우호',
 '적극',
 '수펙스',
 '확보',
 '레론리맙',
 '급락',
 '사업',
 '마련',
 '이점',
 '추천',
 '증가',
 '젬백스젬백스',
 '성민석',
 '럭키석유화학',
 '검토',
 '메틸아민',
 '매직마이크',
 '사대부중',
 '확대',
 '최고급',
 '톈치그룹',
 '우세',
 '확충',
 '란샤오민',
 '씽큐월렛',
 '손춘호',
 '펀인터렉티브',
 '주요한',
 '강점',
 '엔드플레이트',
 '박승영',
 '아벤드',
 '해법',
 '본안심리',
 '텍드림텍',
 '프로그램매매',
 '횡보세',
 '밑돈',
 '남인호',
 '토픽스지수',
 '쇼와덴코',
 '차익',
 '극복',
 '기가와트',
 '본격',
 '약세',
 '증진',
 '씽크빅웅진씽크빅',
 '부속고',
 '시너지',
 '촉진',
 '컬스틴',
 '장단점',
 '엔드플레이트란',
 '밑돌',
 '휴톡스',
 '센티먼트',
 '임예훈',
 '윤재춘',
 '락희화학',
 '의미',
 '듀얼폴드',
 '맥쿼리인프라',
 '웃돈',
 '상쇄하',
 '방안',
 '마그비',
 '텍사트인스',
 '레차그',
 '단점',
 '구혁채',
 '툴리눔',
 '패너마시티비치',
 '웰바이오텍',
 '입관식',
 '기록',
 '중요',
 '취저우시',
 '방법',
 '노영택',
 '광풍',
 '증거보존',
 '감소',
 '활기',
 '특성',
 '루브리컨츠',

In [14]:
b = list(set(list(pos_lexicon_2019[pos_lexicon_2019['cosine_similarity']>=0.7]['news_word'])))

b

['불황',
 '약세',
 '밑돈',
 '셀루메드셀루메드',
 '쎄미켐동진쎄미켐',
 '추천',
 '추진',
 '우세',
 '실패',
 '최고점',
 '단점',
 '툴리눔']

In [15]:
c = list(set(list(pos_lexicon_2019[pos_lexicon_2019['cosine_similarity']>=0.65]['news_word'])))

c

['추진',
 '우세',
 '손춘호',
 '맞소송',
 '콘엘비세미콘',
 '컨설팅하거',
 '불황',
 '펑칭펑',
 '단점',
 '메디콕스메디콕스',
 '툴리눔',
 '패너마시티비치',
 '밑돈',
 '셀루메드셀루메드',
 '웃돌',
 '최고점',
 '감소',
 '적극',
 '약세',
 '쎄미켐동진쎄미켐',
 '추천',
 '증가',
 '럭키석유화학',
 '엔드플레이트란',
 '실패']